In [ ]:
from itertools import starmap
from functools import partial
from collections import namedtuple
import re
import datetime

from IPython import display
from ipywidgets import Button, Text, SelectMultiple, HBox, VBox, Textarea, Image, Label, HTML, Dropdown, IntProgress
import pandas
import jinja2
from pony import orm

import db
db.use_db('minitest1_data/minitest1.sqlite')

In [ ]:
### Javascript kludges to add autocompletion to text boxes and enable multiple selections

autocomplete = jinja2.Template("""
  $( function() {
    var availableTags = [
    {%- for option in options %}
        "{{ option }}",
    {%- endfor %}
    ];
    $( ":input[{{ selector }}]" ).autocomplete({
      minLength: 3,
      source: availableTags
    });
  } );
""")

# Widget definitions

In [ ]:
### Views -- all the individual widgets and UI state

## Immutable app state
with orm.db_session:
    n_submissions = db.Submission.select().count()
    submissions = list(db.Submission.select())
    students = tuple(starmap('{}, {}, {}'.format, orm.select((s.id, s.first_name, s.last_name)
                                                             for s in db.Student)))

## Mutable app state

# the solution that we are pointing to
submission_index = 0

prev_nr = Button(icon='fa-step-backward', description=' previous')
next_nr = Button(icon='fa-step-forward', description=' next')
student_info = Text(placeholder='number, first, last')
email_info = Text(placeholder='email')
signature = Image(format='png', width='100%')

number_progress = IntProgress(
    value=0,
    min=0,
    max=n_submissions,
    step=1,
    description='Progress:',
    bar_style='success',
    orientation='horizontal',
)

# Widget behaviour

In [ ]:
### Controllers  -- hooking views (widgets) up to models

## Utilities

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

def add_option(target, value):
    with target.hold_trait_notifications():
        target.options = target.options + [value]

def load_image(path):
    with open(path, 'rb') as f:
        return f.read()

# rendering
def render_signature():
    global submission_index
    submission = submissions[submission_index]
    with orm.db_session:
        student = db.Submission.get(id=submission.id).student

    # update the UI state
    if student:
        student_info.value = ', '.join([str(student.id), 
                                        student.first_name, 
                                        student.last_name])
        email_info.value = student.email or ''
    else:
        student_info.value = email_info.value = ''

    number_progress.value = submission_index
    
    # Handle submissions without signature images
    try:
        signature.value = load_image(submission.signature_image_path)
    except FileNotFoundError:
        submission_index += 1
        submission_index %= n_submissions
        render_signature()
        


# save the updated data
def update_data():
    # Parse the student entry
    student_nr, first_name, last_name = re.match(r'(\d{7}), (.+), (.+)',
                                                 student_info.value).groups()
    first_name = first_name.strip()
    last_name = last_name.strip()
    email = email_info.value.strip()

    with orm.db_session:
        # Update students if a new student
        stud = db.Student.get(id=int(student_nr)) or\
               db.Student(id=student_nr, first_name=first_name, last_name=last_name)
               
        if email:
            stud.email = email
        # Update submission
        submission = db.Submission.get(id=submissions[submission_index].id)
        submission.student = stud


def save_then_render(callback):
    def _(sender):
        update_data()
        callback(sender)
        render_signature()
    
    return _

## actual controllers - abusing decorator syntax FTW!

@prev_nr.on_click
@save_then_render
def _(sender):
    global submission_index
    submission_index -= 1
    submission_index %= n_submissions


@next_nr.on_click
@save_then_render
def _(sender):
    global submission_index
    submission_index += 1
    submission_index %= n_submissions


@observe(student_info, names="value")
def ensure_format(state):
    info = state['new']
    match = re.match(r'(\d{7}), (.+), (.+)', info)
    if match is None:
        prev_nr.disabled = next_nr.disabled = email_info.disabled = True
    else:
        prev_nr.disabled = next_nr.disabled = email_info.disabled = False

        # also update the email
        with orm.db_session:
            student = db.Student.get(id=int(match.group(1)))

        if student:
            email_info.value = student.email or ''

# Layout

In [ ]:
render_signature()

In [ ]:
HBox([prev_nr, student_info, email_info, next_nr])

In [ ]:
signature

In [ ]:
number_progress

In [ ]:
# apply javascript kludges
# must trigger after the widgets were rendered

selector = "placeholder='{}'".format(student_info.placeholder)
display.display_javascript(display.Javascript(autocomplete.render(selector=selector, options=students)))

In [ ]:
with orm.db_session:
    sub = db.Submission.get(id=44)

In [ ]:
sub

In [ ]:
db.Submission.get(id=1).student